# 04 - Modelling

In [76]:
!pip install keras-tuner -q

In [1]:
import pandas as pd
import numpy as np
import tensorflow as tf
import keras
import matplotlib.pyplot as plt

from sklearn.metrics import classification_report, confusion_matrix, accuracy_score, plot_roc_curve

import shutil
import cv2;

2023-06-21 12:19:57.355515: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
from keras import models, layers, regularizers
from keras.optimizers import Adam, SGD
from keras.callbacks import EarlyStopping
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import train_test_split

from keras.preprocessing.image import ImageDataGenerator;

from keras.applications import ResNet50
from tensorflow.keras.applications.resnet50 import preprocess_input
from keras.preprocessing import image
from keras.models import Model
from keras.layers import Dense, GlobalAveragePooling2D, Input
from keras_tuner import RandomSearch, GridSearch

In [23]:
# Setting random seeds
np.random.seed(132)
tf.random.set_seed(132)

# 1.0 Importing data

In [24]:
# Loading images from preprocessing stage
images = np.load('/Users/chinmayasukumar/Documents/Springboard/Capstone 3 - Metal defect detection/data/interim/images.npy')

In [25]:
# Loading Dataframe
df = pd.read_csv('/Users/chinmayasukumar/Documents/Springboard/Capstone 3 - Metal defect detection/data/interim/data.csv')

In [26]:
df = pd.get_dummies(df, columns=['Type'])

In [27]:
df.head()

,Filename,Number,Type_Crazing,Type_Inclusions,Type_Patches,Type_Pitted,Type_Rolled,Type_Scratches
0,Cr_1.bmp,1,1,0,0,0,0,0
1,Cr_10.bmp,10,1,0,0,0,0,0
2,Cr_100.bmp,100,1,0,0,0,0,0
3,Cr_101.bmp,101,1,0,0,0,0,0
4,Cr_102.bmp,102,1,0,0,0,0,0


In [28]:
# Creating labels, converting to array
labels = df.iloc[:,2:].to_numpy()

In [29]:
labels.shape

(1800, 6)

In [30]:
labels

array([[1, 0, 0, 0, 0, 0],
       [1, 0, 0, 0, 0, 0],
       [1, 0, 0, 0, 0, 0],
       ...,
       [0, 0, 0, 0, 0, 1],
       [0, 0, 0, 0, 0, 1],
       [0, 0, 0, 0, 0, 1]], dtype=uint8)

In [14]:
images.shape

(1800, 200, 200)

#### There are 1800 200x200 images with 6 possible categories corresponding to the type of steel defect

# 2.0 Model selection

## 2.1 Splitting Data

In [31]:
# Reshape images to (1800, 200, 200, 1) to account for black/white channel
images = images.reshape(-1,200,200,1)

In [32]:
# Splitting into train and (test, valid) sets to be split further
X_train, X_test_val, y_train, y_test_val = train_test_split(images, labels, test_size=0.3, random_state=132)

# Splitting (test, valid) set into seperate test and valid sets
X_test, X_val, y_test, y_val = train_test_split(X_test_val, y_test_val, test_size=0.5, random_state=132)

In [17]:
# Checking shapes
(X_train.shape, X_test.shape, X_val.shape)

((1260, 200, 200, 1), (270, 200, 200, 1), (270, 200, 200, 1))

In [35]:
# Checking shapes
(y_train.shape, y_test.shape, y_val.shape)

((1260, 6), (270, 6), (270, 6))

## 2.2 ResNet

In [286]:
# Creating ResNet model
# input_tensor has shape (200, 200, 3) since ResNet only takes RGB images
resnet_model = ResNet50(weights='imagenet', include_top=False, input_tensor=Input(shape=(200, 200, 3)))

### Transforming Grayscale to RGB

In [287]:
# Simulating images so they appear RGB
# np.repeat repeats X_train's last column which is the column added for grayscale
X_train_rgb = np.repeat(X_train, 3, -1)
X_test_rgb = np.repeat(X_test, 3, -1)
X_val_rgb = np.repeat(X_val, 3, -1)

In [288]:
X_train_rgb.shape, X_test_rgb.shape, X_val_rgb.shape

((1260, 200, 200, 3), (270, 200, 200, 3), (270, 200, 200, 3))

### Training ResNet

In [ ]:
x = layers.GlobalAveragePooling2D()(resnet_model.output)


output = layers.Dense(6, activation='softmax')(x)

model = Model(inputs=resnet_model.input, outputs=output)

for layer in resnet_model.layers:
    layer.trainable = False

model.compile(optimizer=Adam(), loss='categorical_crossentropy', metrics=['accuracy'])

model.fit(X_train_rgb, y_train, epochs=10, validation_data=(X_val_rgb, y_val))

Epoch 1/10
40/40 [==============================] - 114s 3s/step - loss: 1.8344 - accuracy: 0.1595 - val_loss: 1.8038 - val_accuracy: 0.1444
Epoch 2/10
40/40 [==============================] - 107s 3s/step - loss: 1.8254 - accuracy: 0.1651 - val_loss: 1.7963 - val_accuracy: 0.1519
Epoch 3/10
40/40 [==============================] - 127s 3s/step - loss: 1.8093 - accuracy: 0.1556 - val_loss: 1.8163 - val_accuracy: 0.1519
Epoch 4/10
40/40 [==============================] - 138s 3s/step - loss: 1.8249 - accuracy: 0.1722 - val_loss: 1.8141 - val_accuracy: 0.1704
Epoch 5/10
40/40 [==============================] - 113s 3s/step - loss: 1.8146 - accuracy: 0.1698 - val_loss: 1.8232 - val_accuracy: 0.1444
Epoch 6/10
40/40 [==============================] - 121s 3s/step - loss: 1.8121 - accuracy: 0.1603 - val_loss: 1.8018 - val_accuracy: 0.1704
Epoch 7/10
40/40 [==============================] - ETA: 0s - loss: 1.8155 - accuracy: 0.1595

### Evaluating ResNet

In [ ]:
# Getting predictions
y_pred_res = model.predict(X_test_rgb)

In [ ]:
# Getting classes of predictions
y_pred_classes = np.argmax(y_pred_res, axis=1)

In [ ]:
# Test classes
y_test_classes = np.argmax(y_test, axis=1)

In [ ]:
# Confusion matrix
print(confusion_matrix(y_test_classes, y_pred_classes))

In [ ]:
# Getting classification report with labels
labels = labels=['Crazing', 'Inclusions', 'Patches', 'Pitted', 'Rolled', 'Scratches']
print(classification_report(y_test_classes, y_pred_classes, target_names=labels, zero_division=True));

#### This model will be put on hold while CNN is tested

## 2.3 CNN

In [19]:
# Creates a CNN model and returns it
def create_model():    
    model = models.Sequential([
            layers.Conv2D(32, (3, 3), activation='relu', input_shape=(200, 200, 1)),
            layers.MaxPooling2D((2, 2)),
            layers.Conv2D(64, (3, 3), activation='relu'),
            layers.MaxPooling2D((2, 2)),
            layers.Conv2D(64, (3, 3), activation='relu'),
            layers.MaxPooling2D((2, 2)),
            layers.Flatten(),
            layers.Dense(64, activation='relu'),
            layers.Dense(6, activation='softmax')
            ])
    
    return model

In [20]:
# Fits model to training data (input) and validates on valuation data
# Callbakcs include patience of 4 and to restore best weights
# Returns history
def train_model(model, X_train, y_train, X_val, y_val):
    model.compile(optimizer=Adam(), loss='categorical_crossentropy', metrics=['accuracy'])
    
    history = model.fit(X_train, y_train, epochs=15, validation_data=(X_val, y_val),\
                                callbacks=[EarlyStopping(patience=4, restore_best_weights=True)])
    return history

In [33]:
# Instantiating base CNN model
base_cnn = create_model()

In [34]:
# Training model 
# Instantiating hist_base_cnn as the history returned from the train_model function
hist_base_cnn = train_model(base_cnn, X_train, y_train, X_val, y_val)

Epoch 1/15
40/40 [==============================] - 84s 2s/step - loss: 1.5803 - accuracy: 0.3056 - val_loss: 1.3718 - val_accuracy: 0.3519
Epoch 2/15
40/40 [==============================] - 68s 2s/step - loss: 0.9392 - accuracy: 0.6317 - val_loss: 0.7641 - val_accuracy: 0.7074
Epoch 3/15
40/40 [==============================] - 78s 2s/step - loss: 0.6895 - accuracy: 0.7373 - val_loss: 0.5428 - val_accuracy: 0.8000
Epoch 4/15
40/40 [==============================] - 79s 2s/step - loss: 0.3950 - accuracy: 0.8706 - val_loss: 0.3556 - val_accuracy: 0.8778
Epoch 5/15
40/40 [==============================] - 80s 2s/step - loss: 0.5125 - accuracy: 0.7968 - val_loss: 0.6789 - val_accuracy: 0.6704
Epoch 6/15
40/40 [==============================] - 90s 2s/step - loss: 0.2997 - accuracy: 0.9008 - val_loss: 0.3009 - val_accuracy: 0.8889
Epoch 7/15
40/40 [==============================] - 72s 2s/step - loss: 0.1965 - accuracy: 0.9310 - val_loss: 0.3032 - val_accuracy: 0.8963
Epoch 8/15
40/40 [==

In [ ]:
# Returns plot of Accuracy and Loss for training and validation sets
def grapher(history, title):
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 5))

    fig.suptitle(title, fontsize=16)

    ax1.plot(history.history['accuracy'])
    ax1.plot(history.history['val_accuracy'])
    ax1.set_title('Model accuracy')
    ax1.set_ylabel('Accuracy')
    ax1.set_xlabel('Epoch')
    ax1.legend(['Train', 'Validation'], loc='upper left')

    ax2.plot(history.history['loss'])
    ax2.plot(history.history['val_loss'])
    ax2.set_title('Model loss')
    ax2.set_ylabel('Loss')
    ax2.set_xlabel('Epoch')
    ax2.legend(['Train', 'Validation'], loc='upper right')

    plt.tight_layout()
    plt.show()

In [ ]:
grapher(hist_base_cnn, 'Base CNN Training progress')

### Evaluation on test set

In [ ]:
y_pred_cnn_base = base_cnn.predict(X_test)

In [ ]:
y_pred_classes = np.argmax(y_pred_cnn_base, axis=1)

In [ ]:
y_test_classes = np.argmax(y_test, axis=1)

In [ ]:
print(confusion_matrix(y_test_classes, y_pred_classes))

In [ ]:
labels = labels=['Crazing', 'Inclusions', 'Patches', 'Pitted', 'Rolled', 'Scratches']
print(classification_report(y_test_classes, y_pred_classes, target_names=labels))

#### Inclusions, Patches, Pits and Scratches aren't classified properly

#### Classes Crazing, Pitted and Scratches don't have very good f1-scores

## 3.0 Hyperparameter tuning

## 3.1  Round 1

In [ ]:
# Creating another CNN for hyperparameter tuning
# Learning rate and optimizers (Adam or SGD) will be chosen as hyperparameters

def create_model_1(hp):
    model = keras.Sequential()
    model.add(layers.Conv2D(32, (3, 3), activation='relu', input_shape=(200, 200, 1)))
    model.add(layers.MaxPooling2D((2, 2)))
    model.add(layers.Conv2D(64, (3, 3), activation='relu'))
    model.add(layers.MaxPooling2D((2, 2)))
    model.add(layers.Conv2D(64, (3, 3), activation='relu'))
    model.add(layers.MaxPooling2D((2, 2)))
    model.add(layers.Flatten())
    model.add(layers.Dense(64, activation='relu'))
    model.add(layers.Dense(6, activation='softmax'))

    # Optimizer can be changed
    optimizer = hp.Choice('optimizer', ['adam', 'sgd'])
    
    # Learning rate can be changed
    learning_rate = hp.Float('learning_rate', 0.0001, 0.01, sampling='log', default=0.001)

    # Conditional statement depending on which optimzier is chosen during Random Search
    if optimizer == 'adam':
        model.compile(optimizer=keras.optimizers.Adam(learning_rate), loss='categorical_crossentropy',\
        metrics=['accuracy'])
    # If the chosen optimizer is SGD, the model will use a SGD with whichever learning rate is chosen
    else:
        model.compile( optimizer=keras.optimizers.SGD(learning_rate),loss='categorical_crossentropy',\
            metrics=['accuracy'])

    return model

In [ ]:
#shutil.rmtree('my_tuner_directory', ignore_errors=True)

# Creating RandomSearch object 
tuner = RandomSearch(create_model_1, objective='val_accuracy', max_trials=10, executions_per_trial=1, \
                     directory='my_tuner_directory', project_name='image_classification')

In [ ]:
tuner.search_space_summary()

In [ ]:
X_train.shape, y_train.shape, X_val.shape, y_val.shape

In [ ]:
# Running Random Search
tuner_hist = tuner.search(X_train, y_train, epochs=10, validation_data=(X_val, y_val), \
             callbacks=[EarlyStopping(patience=3, restore_best_weights=True)])

In [ ]:
tuner.results_summary()

## 3.2 Round 2

#### The best optimizer is Adam with a learning rate >0.00016. 

#### The best performing learning rate in the first search was ~0.0004. The default learning rate of the Adam optimizer is 0.001 so it makes sense to do a Grid Search between ~0.0004 and 0.0015

In [ ]:
# Creating new model with Adam optimizer and tuning for learning rate only
def create_model_2(hp):
    model = keras.Sequential()
    model.add(layers.Conv2D(32, (3, 3), activation='relu', input_shape=(200, 200, 1)))
    model.add(layers.MaxPooling2D((2, 2)))
    model.add(layers.Conv2D(64, (3, 3), activation='relu'))
    model.add(layers.MaxPooling2D((2, 2)))
    model.add(layers.Conv2D(64, (3, 3), activation='relu'))
    model.add(layers.MaxPooling2D((2, 2)))
    model.add(layers.Flatten())
    model.add(layers.Dense(64, activation='relu'))
    model.add(layers.Dense(6, activation='softmax'))

    # Learning rate range for Grid Search is listed below
    learning_rate = hp.Float('learning_rate', 0.000432025255103857, 0.0015, step=0.00007)
    
    model.compile(optimizer=Adam(learning_rate),loss='categorical_crossentropy',\
            metrics=['accuracy'])
    return model

In [ ]:
#shutil.rmtree('my_tuner_directory_2', ignore_errors=True)

# A GridSearch object is now generated 
tuner_2 = GridSearch(create_model_2, objective='val_accuracy', max_trials=15, executions_per_trial=1, \
                     directory='my_tuner_directory_2', project_name='image_classification')

In [ ]:
tuner_2_hist = tuner_2.search(X_train, y_train, epochs=10, validation_data=(X_val, y_val), \
             callbacks=[EarlyStopping(patience=2, restore_best_weights=True)])

In [ ]:
tuner_2.results_summary()

In [ ]:
# Creating dictionary of learning rates and corresponding accuracies
df_dict = {'Learning rate':[0.0007820252551038569, 0.001062025255103857, 0.000922025255103857, 0.001132025255103857,\
                 0.000572025255103857, 0.000992025255103857, 0.000432025255103857, 0.0012720252551038569,\
                 0.0006420252551038569, 0.000502025255103857],
           'Accuracy':[0.9740740656852722, 0.9740740656852722, 0.9592592716217041, 0.9592592716217041, 0.9555555582046509,\
                       0.9555555582046509, 0.9407407641410828, 0.9407407641410828, 0.9370370507240295, 0.9370370507240295]

          };

In [ ]:
# Creating Dataframe on dictionary
hyp_df = pd.DataFrame(df_dict)

In [ ]:
plt.figure(figsize=(8,5))

# Plotting values except for learning rates with highest accuracy
plt.scatter(hyp_df['Learning rate'].iloc[2:], hyp_df['Accuracy'].iloc[2:], color='blue')

# Plotting highest learning rate values
plt.scatter(hyp_df['Learning rate'].iloc[:2], hyp_df['Accuracy'].iloc[:2], color='red')

# Annotating highest values
x1, y1 = hyp_df['Learning rate'][0], hyp_df['Accuracy'][0]
x2, y2 = hyp_df['Learning rate'][1], hyp_df['Accuracy'][1] 

plt.annotate('(' + str(x1.round(5)) + ', \n' + str(y1.round(5)) + ')', [x1+0.00002, y1-0.0025])
plt.annotate('(' + str(x2.round(5)) + ', \n' + str(y2.round(5)) + ')', [x2+0.00002, y2-0.0025])

# Title and labels
plt.title('Accuracies vs. learning rates in trial #2')
plt.xlabel('Learning rate')
plt.ylabel('Accuracy')

plt.show()

## 4.0 Evaluating best models

### Model #1

In [ ]:
# Getting the best modles
models = tuner_2.get_best_models(7)

first, second, third, fourth, fifth, sixth, seventh = models[0], models[1], models[2], models[3], models[4],\
                                                      models[5], models[6]

In [ ]:
# Building a function to predict and evaluate on Test set
labels = ['Crazing', 'Inclusions', 'Patches', 'Pitted', 'Rolled', 'Scratches']

def predictor(model, X_test, y_test):
    y_pred = model.predict(X_test)
    y_pred_classes = np.argmax(y_pred, axis=1)
    y_test_classes = np.argmax(y_test, axis=1)
    print('\n')
    print(confusion_matrix(y_test_classes, y_pred_classes))
    print('\n')
    print(classification_report(y_test_classes, y_pred_classes, target_names=labels))

In [ ]:
y_test_classes.shape

In [ ]:
y_pred.shape

In [ ]:
y_pred_classes.shape

In [ ]:
# Evaluating first model
predictor(first, X_test, y_test)

### Model #2

In [ ]:
# Evaluating second model
predictor(second, X_test, y_test)

### Model #3

In [ ]:
# Evaluating third model
predictor(third, X_test, y_test)

#### The remaining models did not result in better accuracy

#### Second model seems to perform better

## 5.0 Image Tuning

### 4.1 Standard Score Normalizing

In [ ]:
# Subtracting image mean and image standard deviation from each individual image
mean = np.mean(images)
std = np.std(images)

X_adj = (images - mean)/std

In [ ]:
# Checking shape
X_adj.shape, np.array(labels).shape

In [ ]:
# Splitting into train and (test, valid) sets to be split further
X_train_adj, X_test_val_adj, y_train_adj, y_test_val = train_test_split(X_adj, labels, test_size=0.3, random_state=142)

# Splitting (test, valid) set into seperate test and valid sets
X_test_adj, X_val_adj, y_test_adj, y_val_adj = train_test_split(X_test_val_adj, y_test_val, test_size=0.5, random_state=142)

In [ ]:
# Confirming shapes
X_train_adj.shape, y_train_adj.shape

In [ ]:
# Confirming shapes
X_val_adj.shape, y_val_adj.shape

In [ ]:
# Cloning second model to train on normalized data
norm_cnn = models.clone_model(second)

In [ ]:
# Training normalized CNN
history_norm_cnn = train_model(norm_cnn, X_train_adj, y_train_adj, X_val_adj, y_val_adj)

In [ ]:
# Confusion matrix Classification report for Normalized CNN model
predictor(norm_cnn, X_test_adj, y_test_adj)

### 4.2 Rotation 5º

#### Rotating images might result in higher accuracy

In [ ]:
cnn_5 = models.clone_model(second)

In [ ]:
datagen = ImageDataGenerator(rotation_range=5)

datagen.fit(X_train)
cnn_5.compile(loss='categorical_crossentropy', optimizer=Adam(), metrics=['accuracy'])

history_cnn_5 = train_model(cnn_5, X_train_adj, y_train_adj, X_val_adj, y_val_adj)